In [1]:
import csv
import os
import numpy as np
import librosa
import pandas as pd
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras import models, layers
import tensorflow as tf
import matplotlib.pyplot as plt
from pywt import wavedec

2023-10-05 19:10:16.710679: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-10-05 19:10:16.745504: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-10-05 19:10:16.745536: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-10-05 19:10:16.745557: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-10-05 19:10:16.751295: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-10-05 19:10:16.751625: I tensorflow/core/platform/cpu_feature_guard.cc:182] This Tens

In [2]:
# import os
# dataset_path = '../Preprocessing/db6'
# for i, (dirpath, dirnames, filenames) in enumerate(os.walk(dataset_path)):
#     for f in filenames:
#         file_path = os.path.join(dirpath, f)
#         print(file_path)


In [3]:
# header = 'filename'
# header += ' hasil'
# header = header.split()

# file = open('../Preprocessing/hasil.csv', 'w', newline='')
# with file:
#   writer = csv.writer(file)
#   writer.writerow(header)

In [4]:
import os
dataset_path = '../Preprocessing/baru/'
for i, (dirpath, dirnames, filenames) in enumerate(os.walk(dataset_path)):
    for d in dirnames:
      for j, (dirpath2, dirnames2, filenames2) in enumerate(os.walk(os.path.join(dirpath, d))):
        for f in filenames2:
          file_path = os.path.join(dirpath2, f)
          print("Processing file: ", file_path)
          df = pd.read_csv(file_path)

          X = df.drop(['label', 'filename'], axis=1)
          y = df.iloc[:, -1]

          X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

          lb = LabelEncoder()
          X_train = np.expand_dims(X_train, axis=2)
          X_test = np.expand_dims(X_test, axis=2)
          y_test = tf.keras.utils.to_categorical(lb.fit_transform(y_test))
          y_train = tf.keras.utils.to_categorical(lb.fit_transform(y_train))

          def build_model():
            model = models.Sequential(
                [
                    # layers.Conv1D(64, (3), activation="relu", input_shape=(X_train.shape[1], X_train.shape[2])),
                    # layers.MaxPooling1D((3), strides=(2), padding="same"),

                    # layers.Conv1D(32, (3) , activation="relu"),
                    # layers.MaxPooling1D((3), strides=(2), padding="same"),

                    # layers.Flatten(),
                    # layers.Dense(32, activation="relu"),
                    # layers.Dense(5, activation="softmax"),

                    layers.Conv1D(64, (3), activation="relu", input_shape=(X_train.shape[1], X_train.shape[2])),
                    layers.MaxPooling1D((3), strides=(2), padding="same"),

                    layers.Conv1D(32, (3) , activation="relu"),
                    layers.MaxPooling1D((3), strides=(2), padding="same"),

                    layers.Flatten(),
                    layers.Dense(64, activation="relu", activity_regularizer=tf.keras.regularizers.l2(1e-4)),
                    layers.Dense(5, activation="softmax"),
                ]
            )

            return model

          model = build_model()


          optimazer = tf.keras.optimizers.Adam(learning_rate=0.0001)
          model.compile(optimizer=optimazer, loss='categorical_crossentropy', metrics=['acc'])
          history = model.fit(X_train, y_train, epochs=50, batch_size=64, validation_data=(X_test, y_test))
          test_error, test_accuracy = model.evaluate(X_test, y_test, verbose=1)

          to_append = f'{file_path}'
          to_append += f' {test_accuracy}'
          file = open('../Preprocessing/hasil.csv', 'a', newline='')
          with file:
              writer = csv.writer(file)
              writer.writerow(to_append.split())

Processing file:  ../Preprocessing/baru/db1/data_MfccDwtRmsCqtdb1L5.csv
Epoch 1/50
10/10 [==============================] - 1s 32ms/step - loss: 1.8869 - acc: 0.1906 - val_loss: 1.7017 - val_acc: 0.2313
Epoch 2/50
10/10 [==============================] - 0s 9ms/step - loss: 1.5052 - acc: 0.3812 - val_loss: 1.3917 - val_acc: 0.4875
Epoch 3/50
10/10 [==============================] - 0s 8ms/step - loss: 1.2902 - acc: 0.5375 - val_loss: 1.2096 - val_acc: 0.6062
Epoch 4/50
10/10 [==============================] - 0s 8ms/step - loss: 1.1430 - acc: 0.6328 - val_loss: 1.0899 - val_acc: 0.6625
Epoch 5/50
10/10 [==============================] - 0s 9ms/step - loss: 1.0311 - acc: 0.7047 - val_loss: 0.9974 - val_acc: 0.7312
Epoch 6/50
10/10 [==============================] - 0s 12ms/step - loss: 0.9440 - acc: 0.7484 - val_loss: 0.9180 - val_acc: 0.7750
Epoch 7/50
10/10 [==============================] - 0s 12ms/step - loss: 0.8674 - acc: 0.7625 - val_loss: 0.8482 - val_acc: 0.7875
Epoch 8/50
10/1

In [5]:
# dataset_path = '../Preprocessing/db10'
# for i, (dirpath, dirnames, filenames) in enumerate(os.walk(dataset_path)):
#   for f in filenames:
#     file_path = os.path.join(dirpath, f)
#     print('Processing: ', file_path)
#     data = pd.read_csv(file_path)
#     # data.head()

#     X = data.drop(['label', 'filename'], axis=1)
#     y = data.iloc[:, -1]

#     X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

#     lb = LabelEncoder()
#     X_train = np.expand_dims(X_train, axis=2)
#     X_test = np.expand_dims(X_test, axis=2)
#     y_test = tf.keras.utils.to_categorical(lb.fit_transform(y_test))
#     y_train = tf.keras.utils.to_categorical(lb.fit_transform(y_train))

#     X_train = X_train[..., np.newaxis]
#     X_test = X_test[..., np.newaxis]

#     def build_model():
#         model = models.Sequential(
#             [
#                 layers.Conv1D(64, (3), activation="relu", input_shape=(X_train.shape[1], X_train.shape[2])),
#                 layers.MaxPooling1D((3), strides=(2), padding="same"),

#                 layers.Conv1D(32, (3) , activation="relu"),
#                 layers.MaxPooling1D((3), strides=(2), padding="same"),

#                 layers.Flatten(),
#                 layers.Dense(64, activation="relu"),
#                 layers.Dense(5, activation="softmax"),
#             ]
#         )

#         return model

#     model = build_model()


#     optimazer = tf.keras.optimizers.Adam(learning_rate=0.0001)
#     model.compile(optimizer=optimazer, loss='categorical_crossentropy', metrics=['acc'])
#     history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test))


#     # evaluate model on test set
#     test_error, test_accuracy = model.evaluate(X_test, y_test, verbose=1)
#     print("Accuracy on test set is: {}".format(test_accuracy))

#     to_append = f'{file_path}'
#     to_append += f' {test_accuracy}'
#     file = open('../Preprocessing/hasil.csv', 'a', newline='')
#     with file:
#         writer = csv.writer(file)
#         writer.writerow(to_append.split())
